In [1]:
import os
import csv
import json
import sys
import numpy as np

import librosa

In [2]:
# sound_file = "../../databases/TESS_and_RAVDESS/Angry/03-01-05-01-01-01-01.wav"

# # sample rate is per second. 2 second video we have about 44,000 samples
# signal, sr = librosa.load(sound_file, sr=22050)

In [3]:
# mfcc = librosa.feature.mfcc(signal, n_fft=2048, hop_length=512, n_mfcc=13)

In [4]:
# li = [180, -720, 50, -600, -450, -50, -100]
# min_ = -720
# max_ = 180
# li = [2*((x-min_)/(max_-min_))-1 for x in li]
# print(li)

In [5]:
# print(mfcc)
# x = 2*((mfcc-min_)/(max_-min_))-1
# print(x)

In [6]:
sound_emotion_path = "../../databases/TESS_and_RAVDESS/"
data = {
    "mfcc": [],
    "label": []
}

max_ = 0
min_ = 0

In [7]:
def process_sound(filename, dirpath, emotion, sample_rate, n_mfcc, n_fft, hop_length):
    file_path = os.path.join(dirpath, filename)
    # sample rate is per-second, signal.size/sample rate = time
    signal, sr = librosa.load(file_path, sr=sample_rate)
    # Option to round this so we get more samples, but having consistent 1 second samples is important too
    intervals = int(signal.size / sample_rate)
    for x in range(intervals):
        start = x * sample_rate
        stop = start + sample_rate
        mfcc = librosa.feature.mfcc(signal[start:stop], sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc)
        __ = max(max(mfcc.tolist()))
        _ = min(min(mfcc.tolist()))
        max_ = max(__, 0)
        min_ = min(_, 0)

        # Easier to work with
        mfcc = mfcc.T
        data["mfcc"].append(mfcc.tolist())
        data["label"].append(emotion)
        # The length of each mfcc should be equal because we need that in order to properly train on a 1 second-sample with a 
        # sample rate of 22050. If we change anything about the n_fft, or hop_length, we must re-process everything from scratch
    return max_, min_


In [8]:
def save_mfcc(dataset_path, n_mfcc=13, n_fft=1024, hop_length=256, sample_rate=22050):
    # count: 0=Angry, 1=Disgusted, 2=Fearful, 3=Happy, 4=Netural, 5=Sad, 6=Surprised
    for count, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        # print(filenames)
        # print(dirnames)
        print(dirpath)
        if dirpath is not sound_emotion_path:
            if "Angry" in dirpath:
                for f in filenames:
                    max_, min_ = process_sound(f, dirpath, "anger_disgust", sample_rate, n_mfcc, n_fft, hop_length)
            elif "Disgusted" in dirpath:
                for f in filenames:
                    max_, min_ = process_sound(f, dirpath, "anger_disgust", sample_rate, n_mfcc, n_fft, hop_length)
            elif "Suprised" in dirpath:
                for f in filenames:
                    max_, min_ = process_sound(f, dirpath, "surprised_fear", sample_rate, n_mfcc, n_fft, hop_length)
            elif "Fearful" in dirpath:
                for f in filenames:
                    max_, min_ = process_sound(f, dirpath, "surprised_fear", sample_rate, n_mfcc, n_fft, hop_length)
            elif "Happy" in dirpath:
                for f in filenames:
                    max_, min_ = process_sound(f, dirpath, "joy", sample_rate, n_mfcc, n_fft, hop_length)
            elif "Sad" in dirpath:
                for f in filenames:
                    max_, min_ = process_sound(f, dirpath, "sadness", sample_rate, n_mfcc, n_fft, hop_length)
            elif "Neutral" in dirpath:
                for f in filenames:
                    max_, min_ = process_sound(f, dirpath, "neutral", sample_rate, n_mfcc, n_fft, hop_length)
            else:
                continue
    return max_, min_
max_, min_ = save_mfcc(sound_emotion_path)

../../databases/TESS_and_RAVDESS/
../../databases/TESS_and_RAVDESS/Angry
../../databases/TESS_and_RAVDESS/Disgusted
../../databases/TESS_and_RAVDESS/Fearful
../../databases/TESS_and_RAVDESS/Happy
../../databases/TESS_and_RAVDESS/Neutral
../../databases/TESS_and_RAVDESS/Sad
../../databases/TESS_and_RAVDESS/Suprised


In [9]:
print(max_, min_)

57.107505798339844 -600.6202392578125


In [10]:
# probably need to create a new json object 
# for x in data["mfcc"]:
#     arr = np.asarray(x)
#     2*((arr-min_)/(max_-min_))-1
#     data2["mfcc"].append(arr.tolist())
# print(data["mfcc"])

In [11]:
with open("./tess_ravdess.json", "w") as fp:
    json.dump(data, fp, indent=4)